# Tensor Access Pattern Library (`taplib`): An IRON Tool for Data Movements and Tiling

In [ ]:
# Imports
from IPython.display import HTML
import pydoc

## Introduction

Components of *AI Engines* (AIEs) have *Data Movement Accelerators* (DMAs) which are capable of expressing complex on-the-fly data transformations.

The primary purpose of `taplib` is to provide building blocks for expressing *Tensor Access Patterns* (*taps*) used in DMA transformations. To provide a useful utility (and show the usefulness of `taplib`), `taplib` contains a `TensorTiler2D` class which acts as a factory for generating taps for common tiling patterns.

*tiling* is a common type of transformation whereby a larger *image* or *tensor* of data is broken up into smaller *tiles* or groups of tiles. This can be done to encourage smooth streaming behavior, to better conserver resources at specific areas of the memory hierarchy, or because applications or vectorized instructions require data to be structured in a specific way. `TensorTiler2D` provides several methods to generate taps for common tiling patterns.

## Data Transformations in MLIR

Internally, the `mlir-aie` dialect represents tensor access patterns as a combination of an *offset* into a region of data, a set of *strides*, and a set of *sizes*. Most of the DMAs take either 4 dimensions of offsets/sizes/strides or 3 dimensions + a repeat count with a repeat step (which is, for the most part, functionally equivalent to 4 dimensions of sizes/strides).

It takes practice to successfully think about data movements in offsets/sizes/strides; that is why `taplib` provides some tools to help reason about tensor access patterns.

## `taplib`

There are three main classes which are used to help with tiling:

In [ ]:
from aie.helpers.taplib import TensorAccessPattern, TensorAccessSequence, TensorTiler2D

### `TensorAccessPattern` (`tap`)

A `TensorAccessPattern` represents a single set of offset/sizes/strides on a tensor of a particular shape. Let's look at some examples of what you can do with a `TensorAccessPattern`.

In [ ]:
# Create a TensorAccessPattern
tensor_dims = (2, 3)
offset = 4
sizes = [1, 2]
strides = [0, 1]
tap = TensorAccessPattern((2, 3), offset=offset, sizes=[1, 2], strides=[0, 1])

The `TensorAccessPattern` can be visualized in two ways:
- as a heatmap showing the order that elements are accessed
- as a heatmap showing the number of times each element in the tensor is accessed by the `TensorAccessPattern`

In [ ]:
# We can visualize the TensorAccessPattern
tap.visualize(show_arrows=False)

In [ ]:
# We can add some arrows to the visualization, and optionally plot the access count.
tap.visualize(show_arrows=True, plot_access_count=True)

These graphs are based off of *access tensors* which contain either order or count information, respectively.

In [ ]:
access_order, access_count = tap.accesses()
access_order, access_count

There are also methods to get just one of the access tensors, if desired. For larger tensors, it's useful to only calculate what you need to reduce program memory/computation.

In [ ]:
access_order = tap.access_order()
access_count = tap.access_count()

### TensorTileSequence

A TensorTileSequence is a wrapper around a list of tiles. It can be created directly from a list of tiles or it can generate a list of tiles based on functions which produce sizes, strides, or offsets.

In [ ]:
t0 = TensorAccessPattern((8, 8), offset=0, sizes=[1, 1, 4, 4], strides=[0, 0, 8, 1])
t1 = TensorAccessPattern((8, 8), offset=4, sizes=[1, 1, 4, 4], strides=[0, 0, 8, 1])
t2 = TensorAccessPattern((8, 8), offset=32, sizes=[1, 1, 4, 4], strides=[0, 0, 8, 1])

# Create a TensorTileSequence from a list of tiles
taps = TensorAccessSequence.from_taps([t0, t1, t2])

In [ ]:
# You can treat the TensorAccessSequence like a normal python list for common operations
print(taps[0])

print(len(taps))
t3 = TensorAccessPattern((8, 8), offset=36, sizes=[1, 1, 4, 4], strides=[0, 0, 8, 1])
taps.append(t3)
print(len(taps))
print(taps[3])

for t in taps:
    t.visualize()

In [ ]:
# But you can also do some neat things to visualize the tiles in a sequence in one graph
taps.visualize(plot_access_count=True)

In [ ]:
# Or you can visualize the tiles in a sequence in an animation, where each frame of the animation represents a tile in the sequence.
anim = taps.animate()
HTML(anim.to_jshtml())

### TensorTiler2D

While the `TensorAccessSequence` is useful for working with collections of taps, it can still be a bit arduous to create the `TensorAccessPatterns` in the first place.
`TensorTiler2D` is designed to automate the creation of `TensorAccessSequences` for common tiling patterns.

In [ ]:
# This is equivalent to what we created before, but much easier!
tensor_dims = (8, 8)
tile_dims = (4, 4)
simple_tiler = TensorTiler2D.simple_tiler(tensor_dims, tile_dims)
print(len(simple_tiler))
print(simple_tiler[0])

In [ ]:
anim = simple_tiler.animate()
HTML(anim.to_jshtml())

In [ ]:
# There are some additional options available for the simple_tiler, see the arguments below!
# we use the pydoc function instead of help() because it allows the notebook to be tested by CI in a non-interactive way.
print(pydoc.render_doc(TensorTiler2D.simple_tiler, "Help on %s"))

In [ ]:
# There are also more complex tiling patterns, such as groups of tiles
tensor_dims = (16, 16)
tile_dims = (4, 4)
tile_group_dims = (2, 2)
group_tiler = TensorTiler2D.group_tiler(tensor_dims, tile_dims, tile_group_dims)
print(len(group_tiler))
print(group_tiler[0])

In [ ]:
anim = group_tiler.animate()
HTML(anim.to_jshtml())

In [ ]:
# There are some additional options available for the group_tiler, see the arguments below!
# we use the pydoc function instead of help() because it allows the notebook to be tested by CI in a non-interactive way.
print(pydoc.render_doc(TensorTiler2D.group_tiler, "Help on %s"))

In [ ]:
# Most featureful is the step_tiler, which can have non-contiguous groups of tiles
tensor_dims = (32, 32)
tile_dims = (4, 4)
tile_group_dims = (2, 2)
tile_step_dims = (2, 2)
step_tiler = TensorTiler2D.step_tiler(
    tensor_dims, tile_dims, tile_group_dims, tile_step_dims
)
print(len(step_tiler))
print(step_tiler[0])

In [ ]:
anim = step_tiler.animate()
HTML(anim.to_jshtml())

In [ ]:
# There are some additional options available for the step_tiler, see the arguments below!
# we use the pydoc function instead of help() because it allows the notebook to be tested by CI in a non-interactive way.
print(pydoc.render_doc(TensorTiler2D.step_tiler, "Help on %s"))

That concludes the introduction to `taplib`!

In [ ]:
print("Complete!")